# MNIST dataset - Tensorflow
teste da aplicação encontrada no artigo [M. Ponti e G. Costa - Como funciona o Deep Learning](http://conteudo.icmc.usp.br/pessoas/moacir/papers/Ponti_Costa_Como-funciona-o-Deep-Learning_2017.pdf) fazendo uma comparação entre o aprendizado utilizando Machine Learning e Deep Learning em um mesmo dataset, utilizando o tensorflow

## Machine Learning
Como definido por [M. Ponti e G. Costa](http://conteudo.icmc.usp.br/pessoas/moacir/papers/Ponti_Costa_Como-funciona-o-Deep-Learning_2017.pdf),  Machine Learning seriam técnicas de aprendiado mais "rasas" ou "superficiais" (do inglês *shallow*) as quais buscam diretamente por uma única função que possa, a parti de um conjunto de parâmetrôs, gerar o resultado desejado.

$$ f(x) = f_L(... f_2(f_1(x_1))) $$

In [72]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MINST_data/',one_hot=True)

# Variaveis (Matrizes e Vetores)
X = tf.placeholder(tf.float32,[None,784]) # Batch de imagens X
W = tf.Variable(tf.zeros([784,10])) # pesos
b = tf.Variable(tf.zeros([10])) # bias
Y = tf.placeholder(tf.float32, [None,10]) # Classes das Imagens X
inicia = tf.initialize_all_variables()

# modelo que irá gerar as pedrições com base nas imagnes vetorizadas
# modelo de regressão linear: Y = W*X+b
Y_ = tf.nn.softmax(tf.matmul(X, W)+b)

# define função de custo (encropia cruzada)
entropia_cruzada = -tf.reduce_sum(Y * tf.log(Y_ + 0.0001))

# otimização com taxa de aprendizado 0.0025
otimiza = tf.train.GradientDescentOptimizer(0.0025)
treinamento = otimiza.minimize(entropia_cruzada)

sess = tf.Session() # instancia sessão
sess.run(inicia)    # executa sessao e inicializa

# executa 1000 iterações
for i in range(1000):
    # carrega batch de 64 imagens(Y) e suas classes (Y)
    batch_X, batch_Y = mnist.train.next_batch(64)
    dados_treinamento = {X: batch_X, Y:batch_Y}
    
    # treina com o batch atual
    sess.run(treinamento,feed_dict=dados_treinamento)
    # computa entropia-cruzada para acompanhar convergencia
    ce = sess.run(entropia_cruzada,feed_dict=dados_treinamento)

predicao_correta = tf.equal(tf.argmax(Y,1),tf.argmax(Y_,1))
acuracia = tf.reduce_mean(tf.cast(predicao_correta,tf.float32))

print('Acurácia',sess.run(acuracia,feed_dict={X:mnist.test.images, 
                                              Y:mnist.test.labels}),'%')

sess.close()

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz
Acurácia 0.9105 %


## Deep Learning
Já o Deep Learning que é um aprendizado profundo (do inglês *deep*), ou seja, são adicionados "camadas" que possuem seus próprios parâmetros e saídas para realizar a transformação dos dados de entrada, assim aprendendo sucessivas representações dos dadosm intemediárias, assim explica [M. Ponti e G. Costa](http://conteudo.icmc.usp.br/pessoas/moacir/papers/Ponti_Costa_Como-funciona-o-Deep-Learning_2017.pdf)

$$ f_L(... f_2(f_1(x_1,W_1);W_2)...),W_L)$$

In [71]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MINST_data/',one_hot=True)

# Variaveis (Matrizes e Vetores)
X = tf.placeholder(tf.float32,[None,784]) # Batch de imagens X
Y = tf.placeholder(tf.float32, [None,10]) # Classes das Imagens X

# cria e inicializa aleatoriaente os pesos com distribuição normal e sigma=0.1
W1 = tf.Variable(tf.truncated_normal([784,64], stddev=0.1))
# bias são inicializadas com valores fixos 1/10
B1 = tf.Variable(tf.ones([64])/10)

W2 = tf.Variable(tf.truncated_normal([64,32], stddev=0.1))
B2 = tf.Variable(tf.ones([32])/10)

W3 = tf.Variable(tf.truncated_normal([32,10], stddev=0.1))
B3 = tf.Variable(tf.zeros([10]))

inicia = tf.initialize_all_variables()

# modelos das representações intermediárias
X1 = tf.nn.relu(tf.matmul(X,W1)+B1)
X2 = tf.nn.relu(tf.matmul(X1,W2)+B2)

# saída da rede ( a.k.a, logits)
X3 = tf.matmul(X2,W3) + B3
# classificação softmax
Y_ = tf.nn.softmax(X3)

# define função de custo (encropia cruzada)
entr_cruz = -tf.reduce_sum(Y * tf.log(Y_ + 0.0001))

# otimização com taxa de aprendizado 0.0025
otimiza = tf.train.GradientDescentOptimizer(0.0025)
treinamento = otimiza.minimize(entr_cruz)


sess = tf.Session() # instancia sessão
sess.run(inicia)    # executa sessao e inicializa

# carrega batch de 64 imagens(Y) e suas classes (Y)
for k in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(64)
    dados_treinamento = {X: batch_X, Y:batch_Y}
    
    # computa entropia-cruzada para acompanhar convergencia
    sess.run([treinamento,entr_cruz],feed_dict=dados_treinamento)

predicao_correta = tf.equal(tf.argmax(Y,axis=1),tf.argmax(Y_,axis=1))
acuracia = tf.reduce_mean(tf.cast(predicao_correta,tf.float32))

print('Acurácia',sess.run(acuracia,feed_dict={X:mnist.test.images, 
                                              Y:mnist.test.labels}),'%')

sess.close()

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz
Acurácia 0.9746 %
